In [1]:
from monai.utils import first, set_determinism
from monai.transforms import (
    AddChanneld,
    AsChannelFirstd,
    AsDiscrete,
    AsDiscreted,
    Compose,
    EnsureChannelFirstd,
    EnsureTyped,
    EnsureType,
    Invertd,
    Lambdad,
    LoadImaged,
    RandFlipd,
    RandSpatialCropd,
    RandZoomd,
    Resized,
    ScaleIntensityRanged,
    SpatialCrop,
    SpatialCropd,
    ToTensord,
)
from monai.handlers.utils import from_engine
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.metrics import DiceMetric
from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai.data import CacheDataset, DataLoader, Dataset, decollate_batch
from monai.config import print_config
from monai.apps import download_and_extract
import monai.utils as utils

import torch
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
from glob import glob

import itk
from itk import TubeTK as ttk

import numpy as np

import site
site.addsitedir('../../ARGUS')
from ARGUSUtils_Transforms import *

In [2]:
img1_dir = "../../Data/VFoldData/BAMC-PTX*Sliding-Annotations-Linear/"
    
all_images = sorted(glob(os.path.join(img1_dir, '*_?????.nii.gz')))
all_labels = sorted(glob(os.path.join(img1_dir, '*.extruded-overlay-NRS.nii.gz')))

gpu_device = 0

num_classes = 3
class_sliding = 1
class_not_sliding = 2

net_in_dims = 2
net_in_channels = 4
net_channels=(32, 64, 128)
net_strides=(2, 2)
        
num_folds = 15

num_slices = 32
size_x = 320
size_y = 320
roi_size = (size_x,size_y)

num_workers_te = 0
batch_size_te = 1

model_filename_base = "./results/BAMC_PTX_ARUNet-StdDevExtended-ExtrudedRP"

num_images = len(all_images)
print(num_images, len(all_labels))

ns_prefix = ['025ns','026ns','027ns','035ns','048ns','055ns','117ns',
             '135ns','193ns','210ns','215ns','218ns','219ns','221ns','247ns']
s_prefix = ['004s','019s','030s','034s','037s','043s','065s','081s',
            '206s','208s','211s','212s','224s','228s','236s','237s']

fold_prefix_list = []
ns_count = 0
s_count = 0
for i in range(num_folds):
    if i%2 == 0:
        num_ns = 1
        num_s = 1
        if i > num_folds-3:
            num_s = 2
    else:
        num_ns = 1
        num_s = 1
    f = []
    for ns in range(num_ns):
        f.append([ns_prefix[ns_count+ns]])
    ns_count += num_ns
    for s in range(num_s):
        f.append([s_prefix[s_count+s]])
    s_count += num_s
    fold_prefix_list.append(f)
        
train_files = []
val_files = []
test_files = []
for i in range(num_folds):
    tr_folds = []
    for f in range(i,i+num_folds-2):
        tr_folds.append(fold_prefix_list[f%num_folds])
    tr_folds = list(np.concatenate(tr_folds).flat)
    va_folds = list(np.concatenate(fold_prefix_list[(i+num_folds-2) % num_folds]).flat)
    te_folds = list(np.concatenate(fold_prefix_list[(i+num_folds-1) % num_folds]).flat)
    train_files.append(
            [
                {"image": img, "label": seg}
                for img, seg in zip(
                    [im for im in all_images if any(pref in im for pref in tr_folds)],
                    [se for se in all_labels if any(pref in se for pref in tr_folds)])
            ]
        )
    val_files.append(
            [
                {"image": img, "label": seg}
                for img, seg in zip(
                    [im for im in all_images if any(pref in im for pref in va_folds)],
                    [se for se in all_labels if any(pref in se for pref in va_folds)])
            ]
        )
    test_files.append(
            [
                {"image": img, "label": seg}
                for img, seg in zip(
                    [im for im in all_images if any(pref in im for pref in te_folds)],
                    [se for se in all_labels if any(pref in se for pref in te_folds)])
            ]
        )
    print(len(train_files[i]),len(val_files[i]),len(test_files[i]))

62 62
53 4 5
53 5 4
54 4 4
54 4 4
54 4 4
55 4 3
55 3 4
54 4 4
54 4 4
54 4 4
53 4 5
53 5 4
53 4 5
53 5 4
54 4 4


In [3]:
train_shape = itk.GetArrayFromImage(itk.imread(train_files[0][0]["image"])).shape

test_transforms = Compose(
    [
        LoadImaged(keys=["image", "label"]),
        AsChannelFirstd(keys='image'),
        AsChannelFirstd(keys='label'),
        ScaleIntensityRanged(
            a_min=0, a_max=255,
            b_min=0.0, b_max=1.0,
            keys=["image"]),
        SpatialCropd(
            roi_start=[0,0,1],
            roi_end=[320,320,61],
            keys=["image", "label"]),
        Lambdad(
            func=lambda x: np.where(x==3,1,x),
            keys=['label']),
        ARGUS_RandSpatialCropSlicesd(
            num_slices=[num_slices,1],
            axis=0,
            reduce_to_statistics=[True,False],
            extended=True,
            keys=['image', 'label']),
        ToTensord(keys=["image", "label"]),
    ]
)

In [4]:
test_ds = [Dataset(data=test_files[i], transform=test_transforms)
          for i in range(num_folds)]
test_loader = [DataLoader(test_ds[i], batch_size=batch_size_te, num_workers=num_workers_te)
              for i in range(num_folds)]

In [5]:
batchnum = 0
imgnum = 0
lbl = utils.first(test_loader[batchnum])["label"]
m = lbl[imgnum,0,:,:].max()
print(m)
if m == 1:
    img = utils.first(test_loader[0])["image"]
    plt.subplots()
    plt.imshow(img[imgnum,0,:,:])
    plt.subplots()
    plt.imshow(lbl[imgnum,0,:,:])
print("Data Size =", lbl.shape)

tensor(2.)
Data Size = torch.Size([1, 1, 320, 320])


In [6]:
# standard PyTorch program style: create UNet, DiceLoss and Adam optimizer
device = torch.device("cuda:"+str(gpu_device))

In [7]:
def plot_vfold_training_curves(vfold_num, test_loader, min_size_comp, min_portion_comp, p_prior, graph):
    if graph:
        print("   VFOLD =", vfold_num, "of", num_folds)
    
    patient_correct = 0
    patient_incorrect = 0
    patient_false_negatives = 0
    
    frame_correct = 0
    frame_incorrect = 0    
    frame_false_negatives = 0
    
    roi_correct = 0
    roi_incorrect = 0
    roi_false_negatives = 0
    
    loss_file = model_filename_base+"_loss_"+str(vfold_num)+".npy"
    if os.path.exists(loss_file):
        epoch_loss_values = np.load(loss_file)
    
        metric_file = model_filename_base+"_val_dice_"+str(vfold_num)+".npy"
        metric_values = np.load(metric_file)
    
        if graph:
            plt.figure("train", (12, 6))
            plt.subplot(1, 2, 1)
            plt.title("Epoch Average Loss")
            x = [i + 1 for i in range(len(epoch_loss_values))]
            y = epoch_loss_values
            plt.xlabel("epoch")
            plt.plot(x, y)
            plt.ylim([0.2,0.8])
            plt.subplot(1, 2, 2)
            plt.title("Val Mean Dice")
            x = [2 * (i + 1) for i in range(len(metric_values))]
            y = metric_values
            plt.xlabel("epoch")
            plt.plot(x, y)
            plt.ylim([0.2,0.8])
            plt.show()
    
    model_file = model_filename_base+'.best_model.vfold_'+str(vfold_num)+'.pth'
    if os.path.exists(model_file):
        model = UNet(
            dimensions=net_in_dims,
            in_channels=net_in_channels,
            out_channels=num_classes,
            channels=net_channels,
            strides=net_strides,
            num_res_units=2,
            norm=Norm.BATCH,
        ).to(device)    
        model.load_state_dict(torch.load(model_file))
        model.eval()
        with torch.no_grad():
            i = 0
            fname = os.path.basename(test_files[vfold_num][i]["image"])
            prevfname = fname
            frame_roi_count = 0
            frame_roi_count_not_sliding = 0
            patient_frame_count = 0
            patient_frame_count_not_sliding = 0
            for b,test_data in enumerate(test_loader):
                test_outputs = sliding_window_inference(
                    test_data["image"].to(device), roi_size, batch_size_te, model
                )
                for j in range(test_outputs.shape[0]):
                    prevfname = fname
                    fname = os.path.basename(test_files[vfold_num][i]["image"])
                    
                    if fname[:22]!=prevfname[:22]:
                        patient_frame_count += 1
                        if frame_roi_count_not_sliding == 0: # frame_roi_count
                            if graph:
                                print("  * Frame Winner = Sliding ( NS =",frame_roi_count_not_sliding,"of",frame_roi_count,")")
                            if prevfname[3] == 's':
                                frame_correct += 1
                                if graph:
                                    print("    Correct")
                            else:
                                frame_incorrect += 1
                                frame_false_negatives += 1
                                print("    Frame FN =", prevfname)
                        else:
                            patient_frame_count_not_sliding += 1
                            if graph:
                                print("  * Frame Winner = Not Sliding ( NS =",frame_roi_count_not_sliding,"of",frame_roi_count,")")
                            if prevfname[3] == 'n':
                                frame_correct += 1
                                if graph:
                                    print("    Correct")
                            else:
                                frame_incorrect += 1
                                print("    Frame FP =", prevfname)
                        if graph:
                            print()
                            print()
                        frame_roi_count = 0
                        frame_roi_count_not_sliding = 0
                    if fname[:4]!=prevfname[:4]:
                        if patient_frame_count_not_sliding == 0:
                            if graph:
                                print(" *** Patient Winner = Sliding ( NS =",patient_frame_count_not_sliding,"of",patient_frame_count,")")
                            if prevfname[3] == 's':
                                patient_correct += 1
                                if graph:
                                    print("     Correct")
                            else:
                                patient_incorrect += 1
                                patient_false_negatives += 1
                                print("     Patient FN =", prevfname)
                        else:
                            if graph:
                                print(" *** Patient Winner = Not Sliding ( NS =",patient_frame_count_not_sliding,"of",patient_frame_count,")")
                            if prevfname[3] == 'n':
                                patient_correct += 1
                                if graph:
                                    print("     Correct")
                            else:
                                patient_incorrect += 1
                                print("     Patient FP =", prevfname)
                        if graph:
                            print()
                            print()
                    
                    prob_shape = test_outputs[j,:,:,:].shape
                    prob = np.empty(prob_shape)
                    for c in range(num_classes):
                        itkProb = itk.GetImageFromArray(test_outputs[j,c,:,:].cpu())
                        imMathProb = ttk.ImageMath.New(itkProb)
                        imMathProb.Blur(5)
                        itkProb = imMathProb.GetOutput()
                        prob[c] = itk.GetArrayFromImage(itkProb)
                    arrc1 = np.zeros(prob[0].shape)
                    if False:
                        arrc1 = np.argmax(prob,axis=0)
                    else:
                        pmin = prob[0].min()
                        pmax = prob[0].max()
                        for c in range(1,num_classes):
                            pmin = min(pmin, prob[c].min())
                            pmax = min(pmax, prob[c].max())
                        prange = pmax - pmin
                        prob = (prob - pmin) / prange
                        for c in range(num_classes):
                            prob[c] = prob[c] * p_prior[c]
                        arrc1 = np.argmax(prob,axis=0)
                    
                    roi_max_size = np.count_nonzero(test_data["label"][j, 0, :, :].cpu()>0)
                    roi_sliding_min_thresh = max(min_size_comp, roi_max_size*min_portion_comp)
                    
                    itkc1 = itk.GetImageFromArray(arrc1.astype(np.float32))
                    imMathC1 = ttk.ImageMath.New(itkc1)
                    for c in range(num_classes):
                        imMathC1.Erode(10,c,0)
                        imMathC1.Dilate(10,c,0)
                    itkc1 = imMathC1.GetOutputUChar()
                    arrc1 = itk.GetArrayFromImage(itkc1)
                    roi_count_sliding = np.count_nonzero(arrc1==class_sliding)
                    roi_count_not_sliding = np.count_nonzero(arrc1==class_not_sliding)
                    roi_decision = "Unknown"
                    roi_message = "Correct"
                    frame_roi_count += 1
                    if roi_count_sliding>roi_count_not_sliding and roi_count_not_sliding>roi_sliding_min_thresh:
                        roi_decision = "Sliding"
                        if fname[3] == 's':
                            roi_correct += 1
                        else:
                            roi_incorrect += 1
                            roi_false_negatives += 1
                            roi_message = "False Negative"
                    else:
                        frame_roi_count_not_sliding += 1
                        roi_decision = "Not Sliding"
                        if fname[3] == 'n':
                            roi_correct += 1
                        else:
                            roi_incorrect += 1
                            roi_message = "Fales Positive"
                    

                    if graph:
                        print(fname)

                        plt.figure("check", (18, 6))
                        plt.subplot(1, 3, 1)
                        plt.title(f"image {i}")
                        tmpV = test_data["image"][j, 0, :, :]
                        plt.imshow(tmpV, cmap="gray")
                        plt.subplot(1, 3, 2)
                        plt.title(f"label {i}")
                        tmpV = test_data["label"][j, 0, :, :]
                        for c in range(num_classes):
                            tmpV[0,c]=c
                        plt.imshow(tmpV)
                        plt.subplot(1, 3, 3)
                        plt.title(f"output {i}")
                        for c in range(num_classes):
                            arrc1[0,c]=c
                        plt.imshow(arrc1[:,:])
                        plt.show()

                        print("    ROI Number of not-sliding / sliding pixel =", roi_count_not_sliding, roi_count_sliding)
                        print("    ROI Sldiing Min thresh =", roi_sliding_min_thresh)
                        print("    ROI =", roi_decision)
                        print("   ", roi_message)
                        print()
                        print()

                        for c in range(num_classes):
                            arrimg = test_outputs.detach().cpu()[j,c,:,:]
                            itkimg = itk.GetImageFromArray(arrimg)
                            filename = model_filename_base+"_f"+str(vfold_num)+"_i"+str(i)+"_c"+str(c)+".nii.gz"
                            itk.imwrite(itkimg, filename)
                    
                    i += 1
                    
            prevfname = fname          
            patient_frame_count += 1
            if frame_roi_count_not_sliding == 0: # frame_roi_count
                if graph:
                    print("  * Frame Winner = Sliding ( NS =",frame_roi_count_not_sliding,"of",frame_roi_count,")")
                if prevfname[3] == 's':
                    frame_correct += 1
                    if graph:
                        print("    Correct")
                else:
                    frame_incorrect += 1
                    frame_false_negatives += 1
                    print("    Frame FN =", prevfname)
            else:
                patient_frame_count_not_sliding += 1
                if graph:
                    print("  * Frame Winner = Not Sliding ( NS =",frame_roi_count_not_sliding,"of",frame_roi_count,")")
                if prevfname[3] == 'n':
                    frame_correct += 1
                    if graph:
                        print("    Correct")
                else:
                    frame_incorrect += 1
                    print("    Frame FP =", prevfname)
            if graph:
                print()
                print()
            frame_roi_count = 0
            frame_roi_count_not_sliding = 0
            if patient_frame_count_not_sliding == 0:
                if graph:
                    print(" *** Patient Winner = Sliding ( NS =",patient_frame_count_not_sliding,"of",patient_frame_count,")")
                if prevfname[3] == 's':
                    patient_correct += 1
                    if graph:
                        print("     Correct")
                else:
                    patient_incorrect += 1
                    patient_false_negatives += 1
                    print("     Patient FN =", prevfname)
            else:
                if graph:
                    print(" *** Patient Winner = Not Sliding ( NS =",patient_frame_count_not_sliding,"of",patient_frame_count,")")
                if prevfname[3] == 'n':
                    patient_correct += 1
                    if graph:
                        print("     Correct")
                else:
                    patient_incorrect += 1
                    print("     Patient FP =", prevfname)
            if graph:
                print()
                print()
                
    return patient_correct, patient_incorrect, patient_false_negatives, frame_correct, frame_incorrect, frame_false_negatives, roi_correct, roi_incorrect, roi_false_negatives

In [ ]:
min_size = 1000
min_portion = 0.0

for prior in [[1.0,1.0,1.0]]: #[0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6]:
    print('*************')
    print("Prior =", prior)
    t_p_correct = 0
    t_p_incorrect = 0
    t_p_false_negatives = 0
    t_f_correct = 0
    t_f_incorrect = 0
    t_f_false_negatives = 0
    t_r_correct = 0
    t_r_incorrect = 0
    t_r_false_negatives = 0
    for i in range(num_folds):
        (p_correct, p_incorrect, p_false_negatives, f_correct, f_incorrect, f_false_negatives, r_correct, r_incorrect, r_false_negatives) = plot_vfold_training_curves(i, test_loader[i],
            min_size, min_portion, prior, False)
        t_p_correct += p_correct
        t_p_incorrect += p_incorrect
        t_p_false_negatives += p_false_negatives
        t_f_correct += f_correct
        t_f_incorrect += f_incorrect
        t_f_false_negatives += f_false_negatives
        t_r_correct += r_correct
        t_r_incorrect += r_incorrect
        t_r_false_negatives += r_false_negatives
    print()
    print()
    print("Patients: Correct =", t_p_correct, "Incorrect =", t_p_incorrect, "Not Sliding as Sliding =", t_p_false_negatives)
    print("Frame: Correct =", t_f_correct, "Incorrect =", t_f_incorrect, "Not Sliding as Sliding =", t_f_false_negatives)
    print("ROIs: Correct =", t_r_correct, "Incorrect =", t_r_incorrect, "Not Sliding as Sliding =", t_r_false_negatives)
    print('*************')


*************
Prior = [1.0, 1.0, 1.0, 1.0]
     Patient FP = 236s_iimage_1327616672148_clean.nii.gz
    Frame FP = 237s_iimage_24164968068436_CLEAN.nii.gz
     Patient FP = 237s_iimage_24164968068436_CLEAN.nii.gz
    Frame FP = 004s_iimage_73815992352100_clean.nii.gz
    Frame FP = 004s_iimage_74132233134844_clean.nii.gz
     Patient FP = 004s_iimage_74132233134844_clean.nii.gz
    Frame FN = 026ns_image_1083297968960_clean.nii.gz
    Frame FP = 019s_iimage_10705997566592_CLEAN.nii.gz
    Frame FP = 019s_iimage_10891015221417_clean.nii.gz
     Patient FP = 019s_iimage_10891015221417_clean.nii.gz
    Frame FP = 030s_iimage_677741729740_clean.nii.gz
     Patient FP = 030s_iimage_677741729740_clean.nii.gz
    Frame FP = 034s_iimage_3401832241774_clean.nii.gz
     Patient FP = 034s_iimage_3401832241774_clean.nii.gz
     Patient FP = 037s_iimage_588413346180_CLEAN.nii.gz
    Frame FN = 055ns_image_27180764486244_CLEAN.nii.gz
    Frame FP = 043s_iimage_10391571128899_CLEAN.nii.gz
    Frame F